<a href="https://colab.research.google.com/github/shuoscaryo/python_nn/blob/main/number_guesser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
from keras.datasets import mnist

In [ ]:
images = mnist.load_data()

### SHOW imported images

In [ ]:
import matplotlib.pyplot as plt

# Cargar los datos MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Visualizar algunas imágenes
num_images_to_show = 5

for i in range(num_images_to_show):
    # Mostrar la imagen
    plt.imshow(x_train[i], cmap='gray')  # cmap='gray' para mostrar en escala de grises
    plt.title(f'Etiqueta: {y_train[i]}')
    plt.show()

# Nueva sección